In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import openai
from dotenv import load_dotenv

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs

from method.ours.utils import create_driver, get_xpath
from method.ours.parse import parse_form
from method.ours.prompts import get_form_context
from method.ours.history import HistoryTable
from method.ours.constraints import Invalid
from method.ours.generation import (
    generate_constraints_for_input_groups,
    generate_value_for_input_group,
    generate_values_for_input_groups,
    fill_form_with_value_table,
    submit_form
)
from method.ours.feedback import (
    get_local_feedback,
    get_global_feedback
)

In [3]:
constraint_ablation = {
    'context': True,
    'relevant': False,
    'constraints': True,
    # do not edit
    'feedback': True,
}

value_ablation = {
    'context': True,
    'input_group': False,
    'constraints': True,
    # do not edit
    'related': True,
}

In [4]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA' # ['ADA', 'WORD2VEC', 'SPACY']
GRAPH_EMBEDDING_METHOD = 'NODE2VEC' # ['NODE2VEC', 'GCN']

URL = 'https://www.aircanada.com/ca/en/aco/home.html'
# 'https://app.invoicing.co/#/register'
# 'http://localhost:8080/1/common/items/create'
# 'http://localhost:3000/default-channel/en-US/account/register/'
# 'http://localhost:9000/dashboard/discounts/sales/add'
# 'http://localhost:9000/dashboard/customers/add'
# 'https://www.mbta.com/'
# 'https://www.united.com/en/us'
# 'https://www.uhaul.com/Truck-Rentals/'
# 'https://www.cars.com/'
# 'https://www.ups.com/ca/en/Home.page'
# 'https://www.healthgrades.com/find-a-doctor'
# 'https://www.babycenter.com/pregnancy-weight-gain-estimator'
# 'https://www.budget.com/en/home'
# 'https://www.yelp.com/'

In [5]:
def get_to_form(driver):
    try:
        driver.get(URL)
    except:
        print('timeout')
    
    '''
    # AC - Multi-city
    time.sleep(2)
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'bkmgFlights_tripTypeSelector_M'))
    ).click()
    '''
    
    '''
    # Akaunting
    try:
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.ID,
                'email')
            )
        ).send_keys('me@company.com')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.ID,
                'password')
            )
        ).send_keys('password')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//*[@type="submit"]')
            )
        ).click()
    except:
        pass
    '''
    
    '''
    # Saleor
    try:
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[2]/DIV[1]/INPUT[1]')
            )
        ).send_keys('admin@example.com')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[4]/DIV[1]/DIV[1]/INPUT[1]')
            )
        ).send_keys('admin')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[5]/BUTTON[1]')
            )
        ).click()
    except:
        pass
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[2]/MAIN[1]/FORM[1]/DIV[1]/DIV[2]/DIV[4]/DIV[2]/LABEL[1]/SPAN[1]/SPAN[1]/INPUT[1]')
        )
    ).click()
    '''
    
    '''
    # Saleor
    try:
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[2]/DIV[1]/INPUT[1]')
            )
        ).send_keys('admin@example.com')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[4]/DIV[1]/DIV[1]/INPUT[1]')
            )
        ).send_keys('admin')
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[5]/BUTTON[1]')
            )
        ).click()
    except:
        pass
    '''

    '''
    time.sleep(2)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/DIV[1]/DIV[2]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]')
        )
    ).click()
    '''
    
    '''
    # Pet Clinic - Add Owner
    time.sleep(2)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]')
        )
    ).click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]/UL[1]/LI[2]')
        )
    ).click()
    '''
    
    '''
    # UPS - Quote
    time.sleep(2)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'tabs_0_tab_1'))
    ).click()
    '''
    
    '''
    # AC - My Bookings
    time.sleep(2)
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'bkmg-tab-button-mngBook'))
    ).click()'''
    
    '''
    # Budget - Reservation
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, 'PicLoc_value'))
    ).click()
    '''

In [6]:
def find_form():
    return WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]'
        ))
    )

def find_button():
    '''return find_form().find_element(
        By.XPATH,
        '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[2]/MAIN[1]/FORM[1]'
    )'''
    return None
    # return driver.find_elements(By.TAG_NAME, 'button')[10]


In [7]:
driver = create_driver(HEADLESS)
get_to_form(driver)

In [8]:
form = find_form()

In [9]:
history_table = HistoryTable(
    url=URL,
    xpath=get_xpath(driver, form)
)

In [10]:
for element in form.find_elements(By.TAG_NAME, 'input'):
    try:
        element.clear()
    except:
        pass

# Processing

In [11]:
html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
relation_graph, input_groups = parse_form(
    driver,
    form,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/340 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:09<00:00,  3.43it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openai/embeddings_utils.py:68: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


# Generation

In [12]:
value_table = generate_constraints_for_input_groups(
    input_groups,
    ablation_inclusion=constraint_ablation
)

  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant information section. You must choose your constraints in the format of our modified version of the Jest library in JavaScript. The list of functions in this modified format are:\n1. toBeEqual(value) # the input field value is exactly equal to the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondit

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_tripTypeSelector_M')).toBeEqual('M')",
        "role": "assistant"
      }
    }
  ],
  "created": 1690477504,
  "id": "chatcmpl-7gyi839DAagoZaqYRtAG4Jpn7cuDL",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 652,
    "total_tokens": 669
  }
}
  Total running cost: $0.061
toBeEqual ["'M'"]
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_destination_trip_1'))\n.toBeTruthy()\n.toHaveLengthCondition('>', 1)",
        "role": "assistant"
      }
    }
  ],
  "created": 1690477512,
  "id": "chatcmpl-7gyiGolZXKF52RhOh1COgEDahnqkt",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 24,
    "prompt_tokens": 616,
    "total_tokens": 640
  }
}
  Total running cost: $0.120
toBeTruthy ['']
toHaveLengthCondition ["'>'", '1']
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only gener

In [13]:
value_table = generate_values_for_input_groups(
    input_groups,
    value_table,
    ablation_inclusion=value_ablation
)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Vancouver YVR",
        "role": "assistant"
      }
    }
  ],
  "created": 1690477536,
  "id": "chatcmpl-7gyieCcQIAnqwIAvcvwXv8wRby5sG",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 4,
    "prompt_tokens": 344,
    "total_tokens": 348
  }
}
  Total running cost: $0.212
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in qu

In [14]:
'''constraint_count = 0

for key, entry in value_table.entries.items():
    print(key)
    for constraint in entry.constraints:
        print('\t', constraint.to_prompt_string())
        valid = input('is valid: ')
        if valid == '1':
            constraint_count += 1
print(constraint_count)'''

"constraint_count = 0\n\nfor key, entry in value_table.entries.items():\n    print(key)\n    for constraint in entry.constraints:\n        print('\t', constraint.to_prompt_string())\n        valid = input('is valid: ')\n        if valid == '1':\n            constraint_count += 1\nprint(constraint_count)"

In [15]:
fill_form_with_value_table(driver, value_table, input_groups)

<input>value is R</input> at y: (567, 597), x: (219, 319)
R
<input>value is O</input> at y: (567, 597), x: (334, 421)
O
<input>value is M</input> at y: (567, 597), x: (437, 591)
M
<input></input> at y: (574, 594), x: (652, 905)
true
<input></input> at y: (628, 678), x: (198, 466)
Vancouver YVR
<input></input> at y: (628, 678), x: (536, 804)
Los Angeles
<input>DD/MM</input> at y: (628, 678), x: (833, 958)
28/07
<input>DD/MM</input> at y: (628, 678), x: (978, 1093)
28/07


In [16]:
submit_form(driver, input_groups=input_groups, explicit_submit=find_button())

0
1
2
3
4


In [18]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

In [19]:
history_table.add(
    value_table.get_values_dict(),
    'base',
    global_feedback,
    driver.current_url
)

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


# Feedback

In [20]:
new_form = find_form()

In [21]:
new_relation_graph, new_input_groups = parse_form(
    driver,
    new_form,
    prev_relation_graph=relation_graph,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/41 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:05<00:00,  1.73it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openai/embeddings_utils.py:68: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [22]:
global_feedback = get_global_feedback(html, new_html, remove_form_children=True)

In [23]:
global_feedback

[]

In [24]:
value_table = generate_constraints_for_input_groups(
    new_input_groups,
    value_table,
    global_feedback,
    ablation_inclusion=constraint_ablation
)

  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant information section. You must choose your constraints in the format of our modified version of the Jest library in JavaScript. The list of functions in this modified format are:\n1. toBeEqual(value) # the input field value is exactly equal to the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondit

In [25]:
value_table = generate_values_for_input_groups(
    new_input_groups,
    value_table,
    ablation_inclusion=value_ablation
)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

In [26]:
fill_form_with_value_table(driver, value_table, new_input_groups)

<input>value is john.doe@example.com</input> at y: (236, 274), x: (576, 960)
jane.doe@example.com
<input></input> at y: (318, 356), x: (576, 960)
SecurePassword@123
<input></input> at y: (400, 438), x: (576, 960)
SecurePassword@123


In [39]:
submit_form(driver, input_groups=new_input_groups, explicit_submit=find_button())

0
1
2
3
4


In [30]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

history_table.add(
    value_table.get_values_dict(),
    'base',
    global_feedback,
    driver.current_url
)

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


# Constraint Negation

In [17]:
form_context = get_form_context(input_groups)

In [18]:
# history_table.table = history_table.table[:1]

In [19]:
get_attr = lambda x, a, d: x[a] if a in x else d

for field_id, entry in value_table.entries.items():
    input_type = get_attr(entry.input_group.node.element.attrs, 'type', 'text')
    if input_type in ['radio', 'checkbox']:
        continue
    
    for i in range(len(entry.constraints) + 1):
        get_to_form(driver)
        
        value_table_copy = value_table.copy()
    
        if i == 0:
            value_table_copy.entries[field_id].constraints = [Invalid()]
        else:
            value_table_copy.entries[field_id].constraints = [value_table.entries[field_id].constraints[i - 1].copy()]
            value_table_copy.entries[field_id].constraints[0].flip_negative()


        input_group = value_table_copy.entries[field_id].input_group

        value_table_copy = generate_value_for_input_group(
            input_group,
            value_table_copy,
            form_context=form_context
        )

        # fill_form_with_value_table(driver, value_table_copy, input_groups)

        try:
            pass
            # driver.find_element(By.TAG_NAME, 'body').click()
            # submit_form(driver, input_groups=input_groups, explicit_submit=find_button())
        except:
            pass
        
        # time.sleep(1)
        
        new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
        global_feedback = get_global_feedback(html, new_html)
        
        print(global_feedback)
        
        history_table.add(
            value_table_copy.get_values_dict(),
            str(type(value_table_copy.entries[field_id].constraints[0])),
            global_feedback,
            driver.current_url
        )
        
        if len(global_feedback) == 0:
            value_table.entries[field_id].constraints[i - 1].approve()

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

...

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

/Users/parsaalian/Desktop/Research/webform testing project/webform-testing-code/implementation/method/ours/history.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.table = self.table.append({


  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.\nYour decisions must be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.\nOnly generate value and refrain from explaining your answers.\nOnly generate value for the input field in question, and not the ones in the relevant information section.\nWhen generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value \"New York\" to \"NewYork\" to satisfy the constraint not to contain whitespace."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for t

ServiceUnavailableError: The server is overloaded or not ready yet.

In [21]:
'''driver.get(URL)
time.sleep(1)
base_text = driver.find_element(By.TAG_NAME, 'body').text'''

In [20]:
for i, v in enumerate(history_table.table['values']):
    jv = json.loads(v)
    for key, value in jv.items():
        if key not in history_table.table.columns:
            history_table.table[key] = None
        history_table.table.loc[i, key] = value

In [21]:
history_table.table

,values,variation_type,feedback,new_url,//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/FIELDSET[1]/ABC-RADIO-GROUP[1]/DIV[1]/DIV[1]/ABC-RADIO-BUTTON[1]/DIV[1]/INPUT[1],//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/FIELDSET[1]/ABC-RADIO-GROUP[1]/DIV[1]/DIV[1]/ABC-RADIO-BUTTON[2]/DIV[1]/INPUT[1],//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/FIELDSET[1]/ABC-RADIO-GROUP[1]/DIV[1]/DIV[1]/ABC-RADIO-BUTTON[3]/DIV[1]/INPUT[1],//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/AC-SEARCH-TYPE-TOGGLE[1]/DIV[1]/ABC-CHECKBOX[1]/DIV[1]/DIV[1]/INPUT[1],//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/DIV[1]/DIV[1]/ABC-TYPEAHEAD[1]/DIV[1]/ABC-INPUT[1]/ABC-FORM-ELEMENT-CONTAINER[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/INPUT[1],//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/DIV[1]/DIV[1]/ABC-TYPEAHEAD[2]/DIV[1]/ABC-INPUT[1]/ABC-FORM-ELEMENT-CONTAINER[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/INPUT[1],//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/DIV[1]/DIV[1]/ABC-DATE-PICKER[1]/DIV[1]/DIV[1]/ABC-INPUT[1]/ABC-FORM-ELEMENT-CONTAINER[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/INPUT[1],//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/AC-ACOHOME-PAGE[1]/DIV[1]/DIV[1]/AC-BOOKING-MAGNET[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/AC-BKMG-FLIGHTS-TAB[1]/DIV[1]/FORM[1]/FIELDSET[1]/DIV[1]/DIV[1]/DIV[1]/ABC-DATE-PICKER[1]/DIV[1]/DIV[1]/ABC-INPUT[2]/ABC-FORM-ELEMENT-CONTAINER[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/DIV[1]/INPUT[1]
0,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.Inval...,,https://www.aircanada.com/ca/en/aco/home.html,R,O,M,true,123456,Los Angeles,28/07,28/07
1,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.ToBeT...,,https://www.aircanada.com/ca/en/aco/home.html,R,O,M,true,,Los Angeles,28/07,28/07
2,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.ToCon...,Skip to book a flight Skip to main navigation...,https://www.aircanada.com/ca/en/aco/home.html,R,O,M,true,Vancouver,Los Angeles,28/07,28/07
3,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.ToHav...,Skip to book a flight Skip to main navigation...,https://www.aircanada.com/ca/en/aco/home.html,R,O,M,true,YV,Los Angeles,28/07,28/07
4,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.ToHav...,,https://www.aircanada.com/ca/en/aco/home.html,R,O,M,true,"Vancouver International Airport, 3211 Grant Mc...",Los Angeles,28/07,28/07
5,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.ToBeA...,Skip to book a flight Skip to main navigation...,https://www.aircanada.com/ca/en/aco/home.html,R,O,M,true,YVR,Los Angeles,28/07,28/07
6,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraints.concrete.Inval...,Skip to book a flight Skip to main navigation...,https://www.aircanada.com/ca/en/aco/home.html,R,O,M,true,Vancouver YVR,123456,28/07,28/07
7,"{""//BODY/AC-WEB-APP[1]/DIV[1]/MAIN[1]/DIV[1]/A...",<class 'method.ours.constraint

In [49]:
history_table.table = history_table.table[
    history_table.table.feedback.str.len() < len(base_text)
]

In [31]:
set('FEEDBACK-DELIMITER'.join(history_table.table.feedback).split('FEEDBACK-DELIMITER'))

{"Close\nSelect a Nearby Location\n\n\n\n\n\n\n\n\n\n0\n\t\t\t\t\t\t\tCar Rental Locations Found\n\n\n\nThere are no locations for this city. Please enter a new search or click here to browse for a location.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPlease clarify your Budget location\n\n\nWe Can't find the location. Please select from matches below or enter a new search.\n\nPossible Matches",
 "Close\nSelect a Nearby Location\n\n\n\n\n\n\n\n\n\n43\n\t\t\t\t\t\t\tCar Rental Locations Found\n\n\n\n\n\n\n\n\n1\nUnion Station (Amtrak)\n\n\n\n0.5 miles away\n\nAddress: 800 N Alameda St, (Near Cesar Chavez Ave), Los Angeles, US\n\n\nPhone:\n\t\t\t\t\t\t\t\t\t\t(1) 213-617-2977\n\t\t\t\t\t\t\t\t\t\n\nLocation Type:\nCorporate\n\n\n\nHours of Operation: Mon - Fri 8:00 AM - 5:00 PM; Sat 8:00 AM - 1:00 PM\n\t\t\t\t\t\t\t\t\t\n\nSelect This Location\n\n\n\n\n\n\n2\nThe Bloc Shopping Center\n\n\n\n0.9 miles away\n\nAddress: 700 W 7th St - Ste A200, (Inside The Bloc Garage), Los Angeles, US\n\n\nPhone:\n

In [34]:
len(set('FEEDBACK-DELIMITER'.join(history_table.table.feedback).split('FEEDBACK-DELIMITER')))

7

In [23]:
history_table.to_test_case('aoc-multitrip.py')